In [0]:
import json

# Get base parameters
dbutils.widgets.text("bronze_params", "")
dbutils.widgets.text("silver_params", "")

bronze_params = dbutils.widgets.get("bronze_params")
silver_params = dbutils.widgets.get("silver_params")

# Debug: Print the raw input values for troubleshooting
print(f"Raw bronze_params: {bronze_params}")
print(f"Raw silver_params: {silver_params}")

# Parse the JSON string or handle as dictionary for bronze_params
try:
    if isinstance(bronze_params, dict):
        bronze_data = bronze_params
    else:
        bronze_data = json.loads(bronze_params)

    print(f"Parsed bronze_params: {bronze_data}")
except Exception as e:
    print(f"Error parsing bronze_params: {e}")
    raise ValueError("bronze_params is not a valid JSON or dictionary")

# Parse the JSON string or handle as dictionary for silver_params
try:
    if isinstance(silver_params, dict):
        silver_data = silver_params
    else:
        silver_data = json.loads(silver_params)

    print(f"Parsed silver_params: {silver_data}")
except Exception as e:
    print(f"Error parsing silver_params: {e}")
    raise ValueError("silver_params is not a valid JSON or dictionary")

# Access individual variables
start_date = bronze_data.get("start_date", "")
end_date = bronze_data.get("end_date", "")
silver_adls = silver_data.get("silver_adls", "")
gold_adls = silver_data.get("gold_adls", "")

# Debug: Print the extracted values for verification
print(f"Start Date: {start_date}, End Date: {end_date}")
print(f"Silver ADLS Path: {silver_adls}, Gold ADLS Path: {gold_adls}")


In [0]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# Ensure the below library is installed on your fabric environment
import reverse_geocoder as rg
from datetime import date, timedelta

df = spark.read.parquet(silver_output_path).filter(col('time') > start_date)

df = df.limit(100) # added to speed up processings as during testing it was proving a bottleneck
# The problem is caused by the Python UDF (reverse_geocoder) being a bottleneck due to its non-parallel nature and high computational cost per task

def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None
     

# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())
     

# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))
     

# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )
     
# Save the transformed DataFrame to the Silver container
gold_output_path = f"{gold_adls}earthquake_events_gold/"

# Append DataFrame to Silver container in Parquet format
df_with_location_sig_class.write.mode('append').parquet(gold_output_path)